### Collect keypoints 3 - from youtube videos

In [9]:
import os
import cv2
import numpy as np
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip
import constants as cs
import globali as gg
import config_loader as cl
import detect_keypoints as dk
import matplotlib.pyplot as plt

### Extract keypoints from video

In [10]:
def download_and_trim_youtube_video(url, start_time, end_time, action, no_frames):
    
    # load values from settings file 
    gg.actions, gg.counters = cl.read_yaml_fields()
    
    # create directories
    if action not in gg.actions:
        gg.actions.add(action)
        gg.counters[action] = 0
    action_folder = os.path.join(cs.KEYPOINTS_FOLDER, action)
    os.makedirs(action_folder, exist_ok=True)
    action_folder2 = os.path.join(cs.VIDEOS_FOLDER, action)
    os.makedirs(action_folder2, exist_ok=True)
    video_folder = os.path.join(action_folder, str(gg.counters[action]))
    os.makedirs(video_folder, exist_ok=True)
    
    # download youtube video
    yt = YouTube(url)
    video_stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    video_path = video_stream.download(video_folder)
    
    # create file name for the trim video
    name = str(action) + ".mp4"
    trimmed_video_path = os.path.join(video_folder, name)
    
    # cut video
    with VideoFileClip(video_path) as video:
        trimmed_video = video.subclip(start_time, end_time)
        # resized_video = trimmed_video.resize(new_resolution)
        trimmed_video.write_videofile(trimmed_video_path, codec='libx264')
        
    # extract frames from video
    frames = dk.extract_frames(trimmed_video_path, no_frames)  
    
    # delete original video
    os.remove(video_path)
    os.remove(trimmed_video_path)
    
    # Set mediapipe model 
    with dk.mp_holistic.Holistic(min_detection_confidence=0.5,
                                 min_tracking_confidence=0.5) as holistic:
        # for each frame detect keypoints
        for idx, frame in enumerate(frames):
            image, results = dk.mediapipe_detection(frame, holistic)                   # Make detections
            dk.draw_styled_landmarks(image, results)                                   # Draw landmarks 
            #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            #plt.axis('off')
            #plt.show()
            keypoints = dk.extract_keypoints(results)                                  # Extract keypoints
            
            # save keypoints in a file
            npy_folder = os.path.join(video_folder, str(idx))
            np.save(npy_folder, keypoints)
            
            # save video in a file
            frame_path = os.path.join(cs.VIDEOS_FOLDER, action,
                                      str(gg.counters[action]))
            os.makedirs(frame_path, exist_ok=True)
            frame_path = os.path.join(cs.VIDEOS_FOLDER, action, 
                                      str(gg.counters[action]), str(idx) + ".jpg")
            cv2.imwrite(frame_path, frame)
            
        # update actions and counters
        gg.counters[action] += 1
        cl.update_yaml_fields(gg.actions, gg.counters)
    
    print(f"Video tagliato salvato in: {trimmed_video_path}")
    

In [11]:
action = "hello"
url = 'https://www.youtube.com/watch?v=MbggypBmHGw&ab_channel=StartASL'
start_time = 0                                                                  # start time in seconds
end_time = 2                                                                    # end time in seconds
new_resolution = (640, 360)                                                     # new resolution (width, height)

In [12]:
download_and_trim_youtube_video(url, start_time, end_time, action, cs.NUM_FRAME)

RegexMatchError: get_throttling_function_name: could not find match for multiple